In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")




In [10]:
# data ingestion
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://orgzit.com/")
documents = loader.load()


In [11]:
## load data -> docs -> divide into chunks --> vectors embedding--> vector store db
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)


In [16]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()


In [18]:
from langchain_community.vectorstores import FAISS


vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings)

In [19]:
vectorstore

In [25]:
query='Our solution implements your current business processes'
result = vectorstore.similarity_search(query)

print(result[0].page_content)

Don't Change Your Business Processes!

Our solution implements your current business processes, so that you continue with your proven processes and your team does not have to change their way of working.
Customers from these industries have transformed their business with our solutions.





                        HVAC Distributor
                    

                        Optimize operations and accounts and streamline back office management.
                    


                            #Lead, Quoting, and Pricing  
                            #Collaboration  
                            #Order Management  
                        







                        Industrial Equipment
                    

                        Manage Orders, upsell, forecast services and replacements with Orgzit Cloud Platform.


In [31]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [39]:
# Retrieval chain, Document Chain

from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
Answer the follwing based on provided context:
<context>
{context}
</context>

    """
)

documents_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
)

In [40]:
documents_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the follwing based on provided context:\n<context>\n{context}\n</context>\n\n    ')
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'text_inputs': True, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'text_outputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x0000025E43A2AC20>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions

In [43]:
# input --> retrieval chain --> vectorstoredb
retrievar = vectorstore.as_retriever()
from langchain_classic.chains import create_retrieval_chain

chain = create_retrieval_chain(
    retrievar, documents_chain
)


In [44]:
chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025E43A15BA0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the follwing based on provided context:\n<context>\n{context}\n</context>\n\n    ')
            | ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'text_inputs': True, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'text_o

In [47]:
response = chain.invoke({"input":"What are the key features of your product?"})
print(response['answer'])

Based on the provided context, here are some key points about Orgzit:

1. **Product Offerings**: Orgzit offers a flexible B2B Sales CRM with features like lead management, quotation management, dealer management, invoicing, and workflow management. It also provides ready-to-use templates called "Launchers" to help users get started quickly.

2. **Comparison with Other CRMs**: Orgzit is compared with other CRM solutions like Salesforce, Zoho CRM, Hubspot, Pipedrive, and Dynamics 365, indicating its competitive positioning in the CRM market.

3. **Target Industries**: Orgzit caters to various industries, including electrical, chemicals, and IT product distribution, by centralizing information, managing orders, and enhancing customer experiences.

4. **Customer Benefits**: The platform helps businesses streamline operations, manage opportunities, quotes, invoices, inventory, projects, and orders through a centralized dashboard. It aims to double sales and profits by organizing work across